<a href="https://colab.research.google.com/github/beegramin9/Algorithm/blob/main/6_%EC%99%80%EC%9D%B8_%EB%B2%A0%EC%8A%A4%ED%8A%B8_%EB%AA%A8%EB%8D%B8_%EC%9E%90%EB%8F%99%EC%A4%91%EB%8B%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
import pandas as pd
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]
filename

In [ ]:
df = pd.read_csv(filename, header = None)
df.head(3)

In [ ]:
df[12].unique()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:,:-1], df.iloc[:,-1], random_state = seed, stratify = df.iloc[:,-1]
)
X_train.shape, X_test.shape

모델 정의/설정

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Dense(30, input_dim = 12, activation = "relu"))
model.add(Dense(12, activation = "relu"))
model.add(Dense(8, activation = "relu"))
model.add(Dense(1, activation = "sigmoid"))
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

모델 저장 <br>
최솟값 val_loss을 가진 모델을 저장하는 방법

In [ ]:
# 모델 저장 경로
import os
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

In [ ]:
! ls -l

In [ ]:
# 모델 저장 조건 설정
modelpath = MODEL_DIR + "best{epoch:03d}-{val_loss:.4f}.hdf5"

In [ ]:
# ModelCheckpoint: 한 epoch마다 모델의 속성값을 체크
# EarlyStopping: 자동중단, patience만큼 기다렸는데 최솟값이 갱신되지 않으면 stop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
checkpointer = ModelCheckpoint(filepath = modelpath, monitor='val_loss', # 체크할 속성값 acc, val_acc, loss, val_loss
                               verbose = 1,
                               save_best_only = True # 최솟값이 갱신될때만 저장함
                               )
early_stopping = EarlyStopping(monitor='val_loss', patience = 30)

모델 학습 및 저장

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=500, batch_size=200,
          verbose=0, 
          callbacks=[checkpointer, early_stopping] # epoch 한바퀴 돌 때마다 callback함수를 실행함
          )

잘못된 평가

In [ ]:
# best model이 아니기 때문에 잘못된 평가
acc = model.evaluate(X_test, y_test)
print(f"Accuracy: {acc[1]:.4f}")

베스트 모델로 평가

In [ ]:
from tensorflow.keras.models import load_model

best_model = load_model('./model/best246-0.0513.hdf5')
acc = best_model.evaluate(X_test, y_test)
print(f"Accuracy: {acc[1]:.4f}")

시각화

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# acc, val_acc, loss, val_loss
history.history

In [ ]:
y_vloss = history.history['val_loss']
y_acc = history.history['accuracy']
y_vacc = history.history['val_accuracy']
y_loss = history.history['loss']

In [ ]:
x_len = np.arange(len(y_acc))
plt.figure(figsize=(12,8))
plt.plot(x_len, y_vloss, "o", c = "red", markersize=1, label="val_loss")
plt.plot(x_len, y_acc, "o", c = "blue", markersize=1, label="accuracy")
plt.plot(x_len, y_vacc, "o", c = "orange", markersize=1, label="val_accuracy")
plt.plot(x_len, y_loss, "o", c = "green", markersize=1, label="loss")
plt.grid()
plt.legend()
plt.show()